In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import matplotlib.pyplot as plt
from pandasql import sqldf
import sqlite3 
import datetime as dt
import plotly.express as px

## Add rate columns

In [2]:
con = sqlite3.connect('/Users/julianbombard/Desktop/Data Analytics Course/Capstone/database/ATP_Tour')
player_match_stats = pd.read_sql_query('''Select *
                       From player_match_stats''', con)

con.close()

player_match_stats.head()

,player_id,match_id,year,month,player_name,country,height,hand,tourney_name,tourney_level,...,svpt,first_in,first_won,second_won,sv_gms,bp_saved,bp_faced,rank,rank_points,won
0,105357,2015-339_1,2015,1,John Millman,AUS,183,R,Brisbane,A,...,44,24,19,14,8,1,1,153,328,1
1,103813,2015-339_2,2015,1,Jarkko Nieminen,FIN,185,L,Brisbane,A,...,92,59,39,17,14,4,7,73,689,1
2,105902,2015-339_3,2015,1,James Duckworth,AUS,183,R,Brisbane,A,...,45,27,20,11,8,2,3,125,430,1
3,104871,2015-339_4,2015,1,Jeremy Chardy,FRA,188,R,Brisbane,A,...,53,39,31,11,10,0,0,31,1195,1
4,105373,2015-339_5,2015,1,Martin Klizan,SVK,191,L,Brisbane,A,...,130,79,55,27,16,6,8,34,1094,1


In [3]:
player_match_stats['first_sv_pct'] = player_match_stats['first_in'] / player_match_stats['svpt']
player_match_stats['first_sv_win_pct'] = player_match_stats['first_won'] / player_match_stats['first_in']
player_match_stats['second_sv_win_pct'] = player_match_stats['second_won'] / (player_match_stats['svpt'] - player_match_stats['first_in'])
player_match_stats['total_sv_win_pct'] = (player_match_stats['first_won'] + player_match_stats['second_won']) / player_match_stats['svpt']
player_match_stats['bp_save_pct'] = np.where(
    player_match_stats['bp_faced'] == 0, np.nan,
    player_match_stats['bp_saved'] / player_match_stats['bp_faced'])

player_match_stats.head()

,player_id,match_id,year,month,player_name,country,height,hand,tourney_name,tourney_level,...,bp_saved,bp_faced,rank,rank_points,won,first_sv_pct,first_sv_win_pct,second_sv_win_pct,total_sv_win_pct,bp_save_pct
0,105357,2015-339_1,2015,1,John Millman,AUS,183,R,Brisbane,A,...,1,1,153,328,1,0.545455,0.791667,0.700000,0.750000,1.000000
1,103813,2015-339_2,2015,1,Jarkko Nieminen,FIN,185,L,Brisbane,A,...,4,7,73,689,1,0.641304,0.661017,0.515152,0.608696,0.571429
2,105902,2015-339_3,2015,1,James Duckworth,AUS,183,R,Brisbane,A,...,2,3,125,430,1,0.600000,0.740741,0.611111,0.688889,0.666667
3,104871,2015-339_4,2015,1,Jeremy Chardy,FRA,188,R,Brisbane,A,...,0,0,31,1195,1,0.735849,0.794872,0.785714,0.792453,NaN
4,105373,2015-339_5,2015,1,Martin Klizan,SVK,191,L,Brisbane,A,...,6,8,34,1094,1,0.607692,0.696203,0.529412,0.630769,0.750000


In [4]:
player_match_stats['won'] = player_match_stats['won'].astype('bool')
player_match_stats.head()

,player_id,match_id,year,month,player_name,country,height,hand,tourney_name,tourney_level,...,bp_saved,bp_faced,rank,rank_points,won,first_sv_pct,first_sv_win_pct,second_sv_win_pct,total_sv_win_pct,bp_save_pct
0,105357,2015-339_1,2015,1,John Millman,AUS,183,R,Brisbane,A,...,1,1,153,328,True,0.545455,0.791667,0.700000,0.750000,1.000000
1,103813,2015-339_2,2015,1,Jarkko Nieminen,FIN,185,L,Brisbane,A,...,4,7,73,689,True,0.641304,0.661017,0.515152,0.608696,0.571429
2,105902,2015-339_3,2015,1,James Duckworth,AUS,183,R,Brisbane,A,...,2,3,125,430,True,0.600000,0.740741,0.611111,0.688889,0.666667
3,104871,2015-339_4,2015,1,Jeremy Chardy,FRA,188,R,Brisbane,A,...,0,0,31,1195,True,0.735849,0.794872,0.785714,0.792453,NaN
4,105373,2015-339_5,2015,1,Martin Klizan,SVK,191,L,Brisbane,A,...,6,8,34,1094,True,0.607692,0.696203,0.529412,0.630769,0.750000


## Export To SQL

In [5]:
con = sqlite3.connect('/Users/julianbombard/Desktop/Data Analytics Course/Capstone/database/ATP_Tour')

player_match_stats.to_sql('player_match_stats', con, index = False, if_exists = 'replace')

con.close()

## Export to CSV

In [6]:
player_match_stats.to_csv('csvs/player_match_stats.csv', index = False)

Correlation analysis: which stats correlate most with winning matches

In [7]:
stat_columns = player_match_stats.select_dtypes(include = [np.number, 'bool'])

correlations = stat_columns.corr()['won'].drop('won').sort_values(ascending = False)
print(correlations)

total_sv_win_pct     0.612289
first_sv_win_pct     0.510959
second_sv_win_pct    0.464326
bp_save_pct          0.298062
rank_points          0.196311
ace                  0.158939
first_won            0.132647
first_sv_pct         0.117764
second_won           0.109821
height               0.062207
sv_gms               0.026941
player_id            0.007543
year                -0.001317
month               -0.001902
minutes             -0.002389
first_in            -0.018372
svpt                -0.053668
rank                -0.126675
df                  -0.132830
bp_saved            -0.206356
bp_faced            -0.395615
Name: won, dtype: float64


In [8]:
corr_df = correlations.reset_index()
corr_df.columns = ['Statistic', 'Correlation']

fig = px.bar(corr_df, x = 'Statistic', y = 'Correlation', color = 'Correlation', color_continuous_scale = 'Reds')

fig.show()

## Overall Player Analysis

In [9]:
con = sqlite3.connect('/Users/julianbombard/Desktop/Data Analytics Course/Capstone/database/ATP_Tour')
player = pd.read_sql_query('''Select *
                       From players''', con)

con.close()

player.head()

,player_id,player_name,height,hand,avg_minutes,avg_aces,avg_df,avg_svpt,avg_first_in,avg_first_won,avg_second_won,avg_sv_gms,avg_bp_saved,avg_bp_faced,Wins,Losses,Matches_Played,Win_Pct,titles_won
0,100644,Alexander Zverev,198,R,117.14,8.70,3.70,79.10,52.94,39.68,13.21,12.86,3.37,5.40,460,192,652,70.55,23.0
1,102093,Martin Damm,203,L,107.00,14.00,2.00,66.33,43.00,33.00,12.67,10.33,2.00,3.33,2,1,3,66.67,NaN
2,103163,Tommy Haas,188,R,110.22,5.00,4.28,87.50,53.61,37.33,17.89,13.17,5.83,8.50,8,23,31,25.81,NaN
3,103188,Michael Russell,173,R,172.00,4.00,7.00,153.00,96.00,52.00,24.00,18.00,14.00,23.00,0,1,1,0.00,NaN
4,103285,Radek Stepanek,185,R,114.55,5.48,3.93,82.24,49.66,35.52,15.69,12.52,4.86,7.62,20,23,43,46.51,NaN


Filter out players who have played less than 50 matches

In [10]:
player_50 = player[player['Matches_Played'] > 50]

Analyze Potential Correlation Between Aces and Winning matches

In [11]:
fig = px.scatter(player_50, x = 'avg_aces', y = 'Win_Pct', hover_data = 'player_name', color = 'hand', trendline = 'ols', trendline_scope = 'overall', title = 'Do more aces lead to winning?', labels = {
    'avg_aces': 'Aces Per Match',
    'Win_Pct': 'Winning Percentage'
})

fig.show()

In [12]:
correlation = player_50['avg_aces'].corr(player_50['Win_Pct'])
print(f'Correlation is {correlation}')

Correlation is 0.22412878543663256


This suggests a fairly week correlation between hitting more aces and winning more matches

## Tommy Paul before and after his racket switch

I wanted to bring in some metadata to compare some before and afters, and discovered that at the beginning of the 2023 season, American player Tommy Paul switched his racket brand from Wilson to Yonex. I want to compare his statistics, win-loss record, and tournament performances with both rackets. I will use 2021-2022 for the Wilson data, and 2023-2024 for the Yonex data

source: https://www.tennis.com/baseline/articles/tommy-paul-racquet-switch-as-tennis-rules-continue-to-confound-miami

In [13]:
player[player['player_name'] == 'Tommy Paul'] 

,player_id,player_name,height,hand,avg_minutes,avg_aces,avg_df,avg_svpt,avg_first_in,avg_first_won,avg_second_won,avg_sv_gms,avg_bp_saved,avg_bp_faced,Wins,Losses,Matches_Played,Win_Pct,titles_won
507,126205,Tommy Paul,185,R,115.7,5.18,2.29,81.08,49.8,35.41,16.49,12.69,4.11,6.62,176,124,300,58.67,4.0


In [14]:
tommy_paul_matches = player_match_stats[player_match_stats['player_name'] == 'Tommy Paul']

In [15]:
tommy_paul_matches

,player_id,match_id,year,month,player_name,country,height,hand,tourney_name,tourney_level,...,bp_saved,bp_faced,rank,rank_points,won,first_sv_pct,first_sv_win_pct,second_sv_win_pct,total_sv_win_pct,bp_save_pct
2259,126205,2016-0717_282,2016,4,Tommy Paul,USA,185,R,Houston,A,...,3,4,200,262,True,0.641509,0.735294,0.578947,0.679245,0.750000
6093,126205,2017-6116_277,2017,7,Tommy Paul,USA,185,R,Atlanta,A,...,5,9,291,172,True,0.688312,0.679245,0.333333,0.571429,0.555556
6103,126205,2017-6116_289,2017,7,Tommy Paul,USA,185,R,Atlanta,A,...,2,4,291,172,True,0.697917,0.731343,0.482759,0.656250,0.500000
6230,126205,2017-M035_244,2017,7,Tommy Paul,USA,185,R,Washington,A,...,4,6,225,229,True,0.537313,0.833333,0.548387,0.701493,0.666667
6246,126205,2017-M035_273,2017,7,Tommy Paul,USA,185,R,Washington,A,...,4,5,225,229,True,0.676471,0.804348,0.636364,0.750000,0.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48602,126205,2024-560_212,2024,8,Tommy Paul,USA,185,R,US Open,G,...,10,14,14,2985,False,0.603604,0.641791,0.431818,0.558559,0.714286
48687,126205,2024-0329_392,2024,9,Tommy Paul,USA,185,R,Tokyo,A,...,7,10,13,3005,False,0.589474,0.696429,0.589744,0.652632,0.700000
48751,126205,2024-5014_359,2024,10,Tommy Paul,USA,185,R,Shanghai Masters,M,...,6,9,13,3045,False,0.591398,0.727273,0.473684,0.623656,0.666667
48972,126205,2024-0337_359,2024,10,Tommy Paul,USA,185,R,Vienna,A,...,9,11,12,3215,False,0.552941,0.702128,0.473684,0.600000,0.818182


In [16]:
tp_wilson = tommy_paul_matches[tommy_paul_matches['year'].isin([2021, 2022])]
tp_wilson.sample(10)

,player_id,match_id,year,month,player_name,country,height,hand,tourney_name,tourney_level,...,bp_saved,bp_faced,rank,rank_points,won,first_sv_pct,first_sv_win_pct,second_sv_win_pct,total_sv_win_pct,bp_save_pct
17165,126205,2022-0807_290,2022,2,Tommy Paul,USA,185,R,Acapulco,A,...,7,10,39,1342,True,0.570175,0.707692,0.448980,0.596491,0.700000
14886,126205,2021-0429_292,2021,11,Tommy Paul,USA,185,R,Stockholm,A,...,3,4,52,1119,True,0.514706,0.828571,0.454545,0.647059,0.750000
16713,126205,2022-9667_289,2022,1,Tommy Paul,USA,185,R,Adelaide 2,A,...,1,3,41,1374,True,0.684932,0.700000,0.608696,0.671233,0.333333
14902,126205,2021-0438_279,2021,10,Tommy Paul,USA,185,R,Moscow,A,...,6,9,54,1128,True,0.522727,0.760870,0.428571,0.602273,0.666667
14531,126205,2021-0407_291,2021,3,Tommy Paul,USA,185,R,Rotterdam,A,...,0,0,56,1080,True,0.688312,0.867925,0.625000,0.792208,NaN
16723,126205,2022-9667_277,2022,1,Tommy Paul,USA,185,R,Adelaide 2,A,...,0,0,41,1374,True,0.580000,0.827586,0.619048,0.740000,NaN
39136,126205,2021-0407_296,2021,3,Tommy Paul,USA,185,R,Rotterdam,A,...,10,14,56,1080,False,0.604651,0.576923,0.470588,0.534884,0.714286
18926,126205,2022-2807_274,2022,10,Tommy Paul,USA,185,R,Gijon,A,...,0,0,30,1375,True,0.627907,0.851852,0.625000,0.767442,NaN
17951,126205,2022-0311_273,2022,6,Tommy Paul,USA,185,R,Queen's Club,A,...,7,9,35,1113,True,0.688679,0.712329,0.393939,0.613208,0.777778
15374,126205,2021-520_162,2021,5,Tommy Paul,USA,185,R,Roland Garros,G,...,14,18,52,1225,True,0.623529,0.764151,0.515625,0.670588,0.777778


In [17]:
tp_wilson['won'].value_counts()

won
True     62
False    47
Name: count, dtype: int64

In [18]:
tp_yonex = tommy_paul_matches[tommy_paul_matches['year'].isin([2023, 2024])]
tp_yonex.sample(10)

,player_id,match_id,year,month,player_name,country,height,hand,tourney_name,tourney_level,...,bp_saved,bp_faced,rank,rank_points,won,first_sv_pct,first_sv_win_pct,second_sv_win_pct,total_sv_win_pct,bp_save_pct
21703,126205,2023-0329_289,2023,10,Tommy Paul,USA,185,R,Tokyo,A,...,0,0,12,2705,True,0.536585,0.954545,0.789474,0.878049,NaN
48602,126205,2024-560_212,2024,8,Tommy Paul,USA,185,R,US Open,G,...,10,14,14,2985,False,0.603604,0.641791,0.431818,0.558559,0.714286
19475,126205,2023-580_211,2023,1,Tommy Paul,USA,185,R,Australian Open,G,...,4,4,35,1160,True,0.589744,0.804348,0.593750,0.717949,1.000000
23325,126205,2024-0311_300,2024,6,Tommy Paul,USA,185,R,Queen's Club,A,...,2,3,13,2750,True,0.681159,0.765957,0.590909,0.710145,0.666667
45288,126205,2023-0311_282,2023,6,Tommy Paul,USA,185,R,Queen's Club,A,...,10,16,15,2135,False,0.681034,0.582278,0.486486,0.551724,0.625000
23101,126205,2024-0416_294,2024,5,Tommy Paul,USA,185,R,Rome Masters,M,...,12,18,16,2300,True,0.692308,0.543210,0.555556,0.547009,0.666667
45328,126205,2023-0741_300,2023,6,Tommy Paul,USA,185,R,Eastbourne,A,...,3,6,17,2110,False,0.631579,0.708333,0.500000,0.631579,0.500000
48751,126205,2024-5014_359,2024,10,Tommy Paul,USA,185,R,Shanghai Masters,M,...,6,9,13,3045,False,0.591398,0.727273,0.473684,0.623656,0.666667
23486,126205,2024-540_176,2024,7,Tommy Paul,USA,185,R,Wimbledon,G,...,8,11,13,3100,True,0.688742,0.788462,0.574468,0.721854,0.727273
23109,126205,2024-0416_286,2024,5,Tommy Paul,USA,185,R,Rome Masters,M,...,3,5,16,2300,True,0.672414,0.589744,0.736842,0.637931,0.600000


In [19]:
tp_yonex['won'].value_counts()

won
True     80
False    42
Name: count, dtype: int64

In [20]:
tp_both = tommy_paul_matches[tommy_paul_matches['year'].isin([2021, 2022, 2023, 2024])]
tp_both

,player_id,match_id,year,month,player_name,country,height,hand,tourney_name,tourney_level,...,bp_saved,bp_faced,rank,rank_points,won,first_sv_pct,first_sv_win_pct,second_sv_win_pct,total_sv_win_pct,bp_save_pct
14253,126205,2021-0352_255,2021,11,Tommy Paul,USA,185,R,Paris Masters,M,...,1,2,53,1103,True,0.573770,0.742857,0.576923,0.672131,0.500000
14438,126205,2021-0404_219,2021,10,Tommy Paul,USA,185,R,Indian Wells Masters,M,...,0,2,60,1033,True,0.507937,0.812500,0.580645,0.698413,0.000000
14470,126205,2021-0404_260,2021,10,Tommy Paul,USA,185,R,Indian Wells Masters,M,...,0,0,60,1033,True,0.585366,0.833333,0.705882,0.780488,NaN
14490,126205,2021-0404_281,2021,10,Tommy Paul,USA,185,R,Indian Wells Masters,M,...,10,14,60,1033,True,0.598131,0.718750,0.418605,0.598131,0.714286
14520,126205,2021-0407_280,2021,3,Tommy Paul,USA,185,R,Rotterdam,A,...,6,8,56,1080,True,0.701149,0.672131,0.538462,0.632184,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48602,126205,2024-560_212,2024,8,Tommy Paul,USA,185,R,US Open,G,...,10,14,14,2985,False,0.603604,0.641791,0.431818,0.558559,0.714286
48687,126205,2024-0329_392,2024,9,Tommy Paul,USA,185,R,Tokyo,A,...,7,10,13,3005,False,0.589474,0.696429,0.589744,0.652632,0.700000
48751,126205,2024-5014_359,2024,10,Tommy Paul,USA,185,R,Shanghai Masters,M,...,6,9,13,3045,False,0.591398,0.727273,0.473684,0.623656,0.666667
48972,126205,2024-0337_359,2024,10,Tommy Paul,USA,185,R,Vienna,A,...,9,11,12,3215,False,0.552941,0.702128,0.473684,0.600000,0.818182


In [21]:
wl_comp = pd.DataFrame({
    'Yonex' : tp_yonex['won'].map({True: 'Wins', False: 'Losses'}).value_counts(),
    'Wilson' : tp_wilson['won'].map({True: 'Wins', False: 'Losses'}).value_counts()
}).reset_index()

wl_comp

,won,Yonex,Wilson
0,Wins,80,62
1,Losses,42,47


I haven't really figured out how I will handle this one yet, but I need to submit. I will dive deeper into this next week